``` python
# if you running at azure nootbook to prepare env
#!pip install lxml --ignore-installed lxml
#!pip install tushare==1.2.54 
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from greenseer.configuration import do_global_configuration
do_global_configuration("config.properties")

In [3]:
from greenseer.dataset.china_dataset import fetch_default_targets

y = fetch_default_targets()
y.head(5)

,st
code,
002978,0
002981,0
002472,0
002480,0
603105,0


- 600519：茅台

In [4]:
from greenseer.dataset.china_dataset import fetch_multi_report, fetch_train_set
from logging.config import fileConfig
import numpy as np
fileConfig('logging_config.ini')
reports = fetch_multi_report(stock_ids=np.array(["600519","600520"]))
reports.shape

(110, 242)

In [ ]:
train_set,test_set = fetch_train_set()
train_set.shape,test_set.shape